In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:\\Users\\T834432\\Documents\\personnel\\thermostat\\thermostat-agent\\thermostat-292016-05700c0efdbe__.json"



In [2]:
os.environ["PROJECT_ID"] = "thermostat-292016"
os.environ["ACTION_THRESHOLD"] = "1.0"

from google.cloud import storage
#from thermostat import get_metric_list_from_bucket, bucket_name, storage_client
from yadt import parse_date, apply_tz_toronto
from datetime import datetime, timedelta
import pandas as pd
import json
import pickle
import numpy as np



In [9]:
# Instantiates a client
storage_client = storage.Client()
# The name for the new bucket
bucket = storage_client.bucket("thermostat_metric_data")

b = bucket.get_blob('aggregate.p')

pickle_load = b.download_as_bytes()
agg2 = pickle.loads(pickle_load)
agg2

,Indoor Temp.,humidity,PPD,Indoor Temp. Setpoint,MA Temp.,Htg SP,stove_exhaust_temp,Coil Power,Outdoor Temp.,Outdoor RH,...,Occupancy Flag,temp_basement,Sys Out Temp.,dt,action_threshold,heating_state,mpc_action,mpc_indoor_temp_setpoint,mpc_sat_stpt,mpc_sys_out_temp
2020-12-30 08:45:00-05:00,18.732740,51.136543,70.833333,22.0,18.0,22.0,104.208333,8508.164297,-6.870000,62.630000,...,False,20.683333,20.683333,2020-12-30T08:45:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-30 09:00:00-05:00,18.839507,55.150682,67.915556,22.0,18.0,22.0,103.414236,8443.329658,-6.870000,62.630000,...,True,20.985000,20.985000,2020-12-30T09:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-30 09:15:00-05:00,19.105905,54.834244,63.517617,22.0,18.0,22.0,109.701101,8956.625265,-6.564819,61.675078,...,True,21.466667,21.466667,2020-12-30T09:15:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-30 09:30:00-05:00,19.278322,54.348207,60.692157,22.0,18.0,22.0,110.595588,9029.656315,-6.250000,60.690000,...,True,22.080000,22.080000,2020-12-30T09:30:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-30 09:45:00-05:00,19.568141,52.130805,56.350000,22.0,18.0,22.0,103.170673,8423.443779,-5.711538,57.141923,...,False,22.596667,22.596667,2020-12-30T09:45:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-02 19:15:00-05:00,19.727117,49.176727,54.530159,22.0,18.0,22.0,18.062500,0.000000,-5.825873,83.798254,...,False,19.106667,19.106667,2021-01-02T19:15:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-02 19:30:00-05:00,19.578653,47.435057,57.700000,22.0,18.0,22.0,18.062500,0.000000,-6.310000,83.440000,...,False,19.146667,19.146667,2021-01-02T19:30:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-02 19:45:00-05:00,19.483990,46.714990,59.566667,22.0,18.0,22.0,18.145833,0.000000,-5.683333,83.606667,...,False,19.180000,19.180000,2021-01-02T19:45:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-02 19:45:00-05:00,19.483990,46.714990,59.566667,22.0,18.0,22.0,18.145833,0.000000,-5.683333,83.606667,...,False,19.180000,19.180000,2021-01-02T19:45:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
agg2['dup'] = agg2.index.duplicated(keep=False)

In [ ]:
dup = agg2.copy(deep=True)[agg2['dup'] == True]

In [40]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 25)
#dup

In [76]:
agg2 = agg2[~agg2.index.duplicated(keep='first')]
pickle_dump = pickle.dumps(agg2)
b = bucket.get_blob('aggregate.p')
b.temporary_hold = False
b.patch()
b.upload_from_string(data=pickle_dump, content_type='text/plain')
agg2

,Indoor Temp.,humidity,PPD,Indoor Temp. Setpoint,MA Temp.,Htg SP,stove_exhaust_temp,Coil Power,Outdoor Temp.,Outdoor RH,Wind Speed,Wind Direction,Direct Solar Rad.,Occupancy Flag,temp_basement,Sys Out Temp.,dt,action_threshold,heating_state,mpc_action,mpc_indoor_temp_setpoint,mpc_sat_stpt,mpc_sys_out_temp
2020-12-30 08:45:00-05:00,18.732740,51.136543,70.833333,22.0,18.0,22.0,104.208333,8508.164297,-6.870000,62.630000,7.000000,282.690000,38.875000,False,20.683333,20.683333,2020-12-30T08:45:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-30 09:00:00-05:00,18.839507,55.150682,67.915556,22.0,18.0,22.0,103.414236,8443.329658,-6.870000,62.630000,7.000000,282.690000,38.875000,True,20.985000,20.985000,2020-12-30T09:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-30 09:15:00-05:00,19.105905,54.834244,63.517617,22.0,18.0,22.0,109.701101,8956.625265,-6.564819,61.675078,7.275648,282.074715,87.943977,True,21.466667,21.466667,2020-12-30T09:15:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-30 09:30:00-05:00,19.278322,54.348207,60.692157,22.0,18.0,22.0,110.595588,9029.656315,-6.250000,60.690000,7.560000,281.440000,138.562500,True,22.080000,22.080000,2020-12-30T09:30:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-30 09:45:00-05:00,19.568141,52.130805,56.350000,22.0,18.0,22.0,103.170673,8423.443779,-5.711538,57.141923,7.627308,282.343846,133.394231,False,22.596667,22.596667,2020-12-30T09:45:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-01 22:45:00-05:00,19.124153,46.305027,65.847368,18.0,18.0,22.0,18.618421,0.000000,-7.190000,82.578947,5.142632,180.934737,0.000000,True,18.496667,18.496667,2021-01-01T22:45:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 23:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,2021-01-01T23:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 23:15:00-05:00,18.719388,45.835560,72.500000,18.0,18.0,22.0,96.750000,7899.223310,-7.190000,81.750000,5.250000,175.963333,0.000000,False,NaN,NaN,2021-01-01T23:15:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 23:30:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,19.791667,19.791667,2021-01-01T23:30:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
for c in agg2.columns:
    if c.startswith('action_t'):
        agg2.drop(columns=[c], inplace=True)
    if c.startswith('mpc_'):
        agg2.drop(columns=[c], inplace=True)
    if c.startswith('heating_'):
        agg2.drop(columns=[c], inplace=True)

    if c.startswith('action'):
        agg2.drop(columns=[c], inplace=True)
    if c.startswith('indoor_temp'):
        agg2.drop(columns=[c], inplace=True)
    if c.startswith('sat_stpt'):
        agg2.drop(columns=[c], inplace=True)
    if c.startswith('sys_out_temp'):
        agg2.drop(columns=[c], inplace=True)


agg2

,Indoor Temp.,humidity,PPD,Indoor Temp. Setpoint,MA Temp.,Htg SP,stove_exhaust_temp,Coil Power,Outdoor Temp.,Outdoor RH,Wind Speed,Wind Direction,Direct Solar Rad.,Occupancy Flag,temp_basement,Sys Out Temp.,dt
2020-12-30 08:45:00-05:00,18.732740,51.136543,70.833333,22.0,18.0,22.0,104.208333,8508.164297,-6.870000,62.630000,7.000000,282.690000,38.875000,False,20.683333,20.683333,2020-12-30T08:45:00-05:00
2020-12-30 09:00:00-05:00,18.839507,55.150682,67.915556,22.0,18.0,22.0,103.414236,8443.329658,-6.870000,62.630000,7.000000,282.690000,38.875000,True,20.985000,20.985000,2020-12-30T09:00:00-05:00
2020-12-30 09:15:00-05:00,19.105905,54.834244,63.517617,22.0,18.0,22.0,109.701101,8956.625265,-6.564819,61.675078,7.275648,282.074715,87.943977,True,21.466667,21.466667,2020-12-30T09:15:00-05:00
2020-12-30 09:30:00-05:00,19.278322,54.348207,60.692157,22.0,18.0,22.0,110.595588,9029.656315,-6.250000,60.690000,7.560000,281.440000,138.562500,True,22.080000,22.080000,2020-12-30T09:30:00-05:00
2020-12-30 09:45:00-05:00,19.568141,52.130805,56.350000,22.0,18.0,22.0,103.170673,8423.443779,-5.711538,57.141923,7.627308,282.343846,133.394231,False,22.596667,22.596667,2020-12-30T09:45:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-01 22:45:00-05:00,19.124153,46.305027,65.847368,18.0,18.0,22.0,18.618421,0.000000,-7.190000,82.578947,5.142632,180.934737,0.000000,True,18.496667,18.496667,2021-01-01T22:45:00-05:00
2021-01-01 23:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,2021-01-01T23:00:00-05:00
2021-01-01 23:15:00-05:00,18.719388,45.835560,72.500000,18.0,18.0,22.0,96.750000,7899.223310,-7.190000,81.750000,5.250000,175.963333,0.000000,False,NaN,NaN,2021-01-01T23:15:00-05:00
2021-01-01 23:30:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,19.791667,19.791667,2021-01-01T23:30:00-05:00


In [75]:
from itertools import chain

current_date = parse_date("2021-01-01T23:45:00-0500")
mpc = {
    "action_threshold": 1.0,
    "heating_state": False,
    "mpc_action": 0.24015000020290925,
    "mpc_indoor_temp": 18.63394,
    "mpc_indoor_temp_setpoint": 18.0,
    "mpc_occupancy_flag": 0,
    "mpc_sat_stpt": 0.24015000020290925,
    "mpc_sys_out_temp": 19.48333333333333
}


del mpc["mpc_indoor_temp"]
del mpc["mpc_occupancy_flag"]


#decision = {
#            "heating_state" : None,
#            "action_threshold" : None
#    }

#concat = dict(chain.from_iterable(d.items() for d in (mpc, decision)))

agg_mpc = pd.DataFrame(mpc, index=[current_date])

agg2 = pd.concat([agg2, agg_mpc.reindex(agg2.index)], axis=1)
#agg2.update(agg_mpc)
agg2


,Indoor Temp.,humidity,PPD,Indoor Temp. Setpoint,MA Temp.,Htg SP,stove_exhaust_temp,Coil Power,Outdoor Temp.,Outdoor RH,Wind Speed,Wind Direction,Direct Solar Rad.,Occupancy Flag,temp_basement,Sys Out Temp.,dt,action_threshold,heating_state,mpc_action,mpc_indoor_temp_setpoint,mpc_sat_stpt,mpc_sys_out_temp
2020-12-30 08:45:00-05:00,18.732740,51.136543,70.833333,22.0,18.0,22.0,104.208333,8508.164297,-6.870000,62.630000,7.000000,282.690000,38.875000,False,20.683333,20.683333,2020-12-30T08:45:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-30 09:00:00-05:00,18.839507,55.150682,67.915556,22.0,18.0,22.0,103.414236,8443.329658,-6.870000,62.630000,7.000000,282.690000,38.875000,True,20.985000,20.985000,2020-12-30T09:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-30 09:15:00-05:00,19.105905,54.834244,63.517617,22.0,18.0,22.0,109.701101,8956.625265,-6.564819,61.675078,7.275648,282.074715,87.943977,True,21.466667,21.466667,2020-12-30T09:15:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-30 09:30:00-05:00,19.278322,54.348207,60.692157,22.0,18.0,22.0,110.595588,9029.656315,-6.250000,60.690000,7.560000,281.440000,138.562500,True,22.080000,22.080000,2020-12-30T09:30:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-30 09:45:00-05:00,19.568141,52.130805,56.350000,22.0,18.0,22.0,103.170673,8423.443779,-5.711538,57.141923,7.627308,282.343846,133.394231,False,22.596667,22.596667,2020-12-30T09:45:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-01 22:45:00-05:00,19.124153,46.305027,65.847368,18.0,18.0,22.0,18.618421,0.000000,-7.190000,82.578947,5.142632,180.934737,0.000000,True,18.496667,18.496667,2021-01-01T22:45:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 23:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,2021-01-01T23:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 23:15:00-05:00,18.719388,45.835560,72.500000,18.0,18.0,22.0,96.750000,7899.223310,-7.190000,81.750000,5.250000,175.963333,0.000000,False,NaN,NaN,2021-01-01T23:15:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 23:30:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,19.791667,19.791667,2021-01-01T23:30:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def get_metric_from_bucket(blob):

    last_json = []
    
    try:
        json_str = blob.download_as_bytes()
        j = json.loads(json_str)
        #j['dt'] = apply_tz_toronto(datetime.fromtimestamp(j['timestamp']))
        
        #if len(list(j.keys())) > 0:
        #    if 'sound' in j.keys():
        #        del j['sound']
        #    if 'timestamp'
        #    del j['timestamp']
        if isinstance(j, list):
            for item in j:
                last_json.append(item)
        else:
            last_json.append(j)
            
    except json.JSONDecodeError:
        pass

    return last_json

In [ ]:
###
###
###
###
bucket_name = "thermostat_metric_data"
end_date = "2020-12-31T00:00:00-0500"
end_date = parse_date(end_date)
metric_list = list(storage_client.list_blobs(bucket_name, prefix='thermostat'))
metric_list.reverse()

In [ ]:
def date_selection_realtime(dt):
    selection_end = end_date

    select = True
    end = False
    if dt < selection_end:
        #print(dt)
        select = False
        end = True

    return select, end

In [ ]:
def aggregator(blob_list, date_function, value_function, date_select_function):
    aggregation = pd.DataFrame()

    for m in blob_list:
        metric_json = get_metric_from_bucket(m) 
        print(m.name)
        for item in metric_json:

            #need data from filename
            if m.name.startswith('environment_sensor_basement-'):
                item['location'] = 'house.basement'
                str_date = m.name.replace('environment_sensor_basement-','')
                item['dt'] = parse_date(str_date)
            
            date_function(item)
            metric_dict = value_function(item)
            
            select, end = date_select_function(metric_dict['dt'])
            if select:
                df = pd.DataFrame(metric_dict,index=[metric_dict['dt']])
                aggregation = aggregation.append(df)

            if end:
                break
        if end:
            break
            
    return aggregation

def value_function_thermostat(i):
    if 'sound' in i.keys():
        del i['sound']
    if 'timestamp' in i.keys():
        del i['timestamp']
    return i

def date_function_thermostat(i):
    i['dt'] = apply_tz_toronto(datetime.fromtimestamp(i['timestamp']))


In [ ]:

thermostat_agg = aggregator(metric_list, date_function_thermostat, value_function_thermostat, date_selection_realtime)
pickle.dump( thermostat_agg, open( "thermostat_agg.p", "wb" ) )
thermostat_agg

In [ ]:
def create_file(payload, filename):
    """Create a file.

  The retry_params specified in the open call will override the default
  retry params for this particular file handle.

  Args:
    filename: filename.
  """
    blob = bucket.blob(filename)

    blob.upload_from_string(data=payload,
                            content_type='text/plain')





In [ ]:
# Instantiates a client
#storage_client = storage.Client()

# The name for the new bucket
#bucket_name = "thermostat_metric_data"
#bucket = storage_client.bucket(bucket_name)

#pickle_dump = pickle.dumps(agg2)
#create_file(pickle_dump, 'aggregate.p')

In [ ]:
pickle.dump( thermostat_agg, open( "thermostat_agg.p", "wb" ) )
thermostat_agg

In [ ]:
bucket_climacell = "climacell_data"
realtime_list = list(storage_client.list_blobs(bucket_climacell, prefix='realtime'))
realtime_list.reverse()

def date_function_climacell(i):
    #print(i)
    i['dt'] = parse_date(i['observation_time']['value'])
    del i['observation_time']
    #"observation_time": {"value": "2020-10-18T21:15:02.777Z"}}¸

def value_function_climacell(i):
    #print(i)
    field = ['temp','humidity','wind_speed','wind_direction','surface_shortwave_radiation']
    if 'observation_time' in i.keys():
        field.append('observation_time')
    else:
        field.append('dt')

    value_dict = {}

    for f in field:
        if f != 'dt':
            value_dict[f] = i[f]['value']
        else:
            value_dict[f] = i[f]

    return value_dict


In [ ]:
###
###
###
realtime_agg = aggregator(realtime_list, date_function_climacell, value_function_climacell, date_selection_realtime)
realtime_agg

In [ ]:
def rename_climacell_columns(data):
    data.rename(columns={'humidity':'Outdoor RH', 
                                'temp':'Outdoor Temp',
                                'surface_shortwave_radiation':'Direct Solar Rad.',
                                'wind_speed':'Wind Speed',
                                'wind_direction':'Wind Direction'}, inplace = True)
rename_climacell_columns(realtime_agg)

In [ ]:

stove = thermostat_agg.copy(deep=True)[thermostat_agg['location'] == 'house.basement.stove'][['dt','stove_exhaust_temp']]
t_a = thermostat_agg.copy(deep=True)[thermostat_agg['location'] != 'house.basement.stove']
t_a.sort_values('dt', inplace=True)
stove.sort_values('dt', inplace=True)
del t_a['stove_exhaust_temp']
del t_a['location']
agg = pd.merge_asof(t_a, stove,left_on='dt',right_on='dt',direction="nearest")
agg.set_index('dt', inplace=True)
agg



In [ ]:
agg.sort_values('dt',inplace=True)
realtime_agg.sort_values('dt',inplace=True)
agg = pd.merge_asof(agg, realtime_agg,left_on='dt',right_on='dt',direction="nearest")
agg.set_index('dt', inplace=True)
agg = agg[~agg.index.duplicated(keep='first')]
#agg['dup'] = agg.index.duplicated(keep=False)
#agg = agg[agg['dup']]
#agg.sort_values('dt',inplace=True)
agg

In [ ]:
####
####
####
#agg2 = agg.copy(deep=True).resample('15Min').interpolate(method='linear')
#m.set_index('dt', inplace=True)
#agg2 = agg.copy(deep=True).resample('15Min').mean()
#agg2

In [ ]:
end_index = agg.index.max()
start_index = agg.index[agg2.index.searchsorted(end_index - pd.Timedelta(value=3, unit='hours'))]
print(start_index)
print(end_index)
agg_motion = agg.copy()[start_index:end_index]
#agg_motion


m = agg_motion[['motion']]
m['motion'] = m['motion'].apply(lambda x: 1 if x else 0)
m = m.resample('3min').sum()
# Normalize values
m['motion'] = (m['motion']-m['motion'].min())/(m['motion'].max()-m['motion'].min())
# Exponential decay
m['motion'] = m['motion'].ewm(halflife='6Min', times=m.index).mean()
m = m[['motion']].resample('15min').sum()
# Values bellow 50 quantile will be False
m['quantile'] = m['motion'].quantile(q=0.50)
#m['motion'] = m.apply(lambda x: True if x['motion'] > x['quantile'] else False, axis=1)
#del m['quantile']
m

In [ ]:
####
####
####
agg2 = agg2.merge(m, left_index=True, right_index=True)
agg2

In [ ]:

hourly_start = agg2.index.max().to_pydatetime()
hourly_end = hourly_start + timedelta(hours=4)

def date_selection_hourly(dt):
    select = False
    end = False

    
    if dt <= hourly_end and dt >= hourly_start:
        select = True

    if dt > hourly_end:
        end = True

    return select, end

hourly_list = list(storage_client.list_blobs(bucket_climacell, prefix='hourly'))
hourly_list.reverse()



hourly_agg = aggregator(hourly_list, date_function_climacell, value_function_climacell, date_selection_hourly)
rename_climacell_columns(hourly_agg)
hourly_agg

In [ ]:
agg2

In [ ]:

def get_metric_from_bucket(blob):

    last_json = []

    try:
        json_str = blob.download_as_bytes()
        j = json.loads(json_str)
        #j['dt'] = apply_tz_toronto(datetime.fromtimestamp(j['timestamp']))

        #if len(list(j.keys())) > 0:
        #    if 'sound' in j.keys():
        #        del j['sound']
        #    if 'timestamp'
        #    del j['timestamp']
        if isinstance(j, list):
            for item in j:
                last_json.append(item)
        else:
            last_json.append(j)

    except json.JSONDecodeError:
        pass

    return last_json


def aggregator(blob_list, date_function, value_function, date_select_function, end_date, start_date=None):
    aggregation = pd.DataFrame()

    for m in blob_list:
        metric_json = get_metric_from_bucket(m)
        print(m.name)
        for item in metric_json:

            #need data from filename
            if m.name.startswith('environment_sensor_basement-'):
                item['location'] = 'house.basement'
                str_date = m.name.replace('environment_sensor_basement-','')
                item['dt'] = parse_date(str_date, toronto=True)

            date_function(item)
            metric_dict = value_function(item)

            select, end = date_select_function(metric_dict['dt'], end_date, start_date)
            if select:
                df = pd.DataFrame(metric_dict, index=[metric_dict['dt']])
                aggregation = aggregation.append(df)

            if end:
                break
        if end:
            break

    return aggregation


# Instantiates a client
storage_client = storage.Client()
# The name for the new bucket
bucket = storage_client.bucket("thermostat_metric_data")

basement = agg2.index.min().to_pydatetime()


if pd.isnull(basement):
    end_date = "2020-12-28T12:00:00-0500"
    end_date = parse_date(end_date)
else:
    end_date = basement

print("end_date : {}".format(end_date))

def date_selection_realtime(dt, end_date, start_date):
    selection_end = end_date

    select = True
    end = False
    if dt < selection_end:
        #print(dt)
        select = False
        end = True

    return select, end

def value_function_basement(i):

    i['temp_basement'] = i['temperature']
    del i['temperature']
    del i['original_payload']

    return i


def date_function_basement(i):
    return

basement_list = list(storage_client.list_blobs(bucket, prefix='environment_sensor_basement-'))
basement_list.reverse()

basement_agg = aggregator(basement_list, date_function_basement,
                        value_function_basement,
                        date_selection_realtime,
                        end_date)

basement_agg = basement_agg.resample('15Min').mean()
basement_agg


In [ ]:
basement_agg = basement_agg.resample('15Min').mean()
#del agg2['temp_basement']
agg2 = agg2.merge(basement_agg, left_index=True, right_index=True)
agg2

In [ ]:
pickle_dump = pickle.dumps(agg2)
b = bucket.get_blob('aggregate.p')
b.temporary_hold = True
b.patch()
b.upload_from_string(data=pickle_dump, content_type='text/plain')

In [ ]:
thermostat_agg = pickle.load( open( "thermostat_agg.p", "rb" ) )
thermostat_agg

In [ ]:

thermthermostat_agg = thermostat_agg.merge(basement_agg, left_index=True, right_index=True)

In [ ]:
thermthermostat_agg

In [ ]:
agg2 = agg2.replace({np.nan: None})
agg2['dt'] = agg2.index.values
#agg2['dt'] = agg2['dt'].apply(lambda x: x.to_pydatetime().isoformat())
agg2

In [ ]:
agg2

In [ ]:

basement_tmp = agg2.copy(deep=True)[agg2['temp_basement']][['temp_basement']]
